---
title: Randomized SVD
description: Randomized singular value decomposition algorithm for efficient low-rank matrix approximation
keywords: [randomized SVD, singular value decomposition, low-rank approximation, matrix factorization, dimensionality reduction, random projection]
numbering:
  equation:
    enumerator: 5.%s
    continue: true
  proof:theorem:
    enumerator: 5.%s
    continue: true
  proof:algorithm:
    enumerator: 5.%s
    continue: true
  proof:definition:
    enumerator: 5.%s
    continue: true
  proof:proposition:
    enumerator: 5.%s
    continue: true
---


:::{prf:algorithm} Randomized SVD
:label: alg-randomized-svd

**Input:** Matrix $A \in \R^{m \times n}$, target rank $k$, oversampling parameter $p$

1. Sample a random Gaussian matrix $\vec{\Omega} \in \R^{n \times (k+p)}$
1. Compute $\vec{Y} = \vec{A} \vec{\Omega}$
1. Compute QR factorization $\vec{Q}\vec{R} = \Call{qr}(\vec{Y})$
1. Compute $\vec{X} = \vec{Q}^\T \vec{A}$
1. Compute SVD $\vec{U}'\vec{\Sigma}\vec{V} = \Call{svd}(\vec{X})$
1. Set $\vec{U} = \vec{Q}\vec{U}'$

**Output:** $\vec{U} \vec{\Sigma} \vec{V}^\T$ 
:::


:::{prf:theorem}
For any $p\geq 2$, 
\begin{equation*}
\EE\left[ \| \vec{A} - \vec{U}\vec{\Sigma}\vec{V}^\T \|_\F^2 \right] \leq \left( 1 + \frac{k}{p-1} \right) \| \vec{A} - \llbracket \vec{A} \rrbracket_k \|_\F^2 .
\end{equation*}
:::

:::{admonition} Proof
:class: dropdown

TODO
:::

## Numerical Example